# Enterprise RAG

## สิ่งที่จะได้รู้
- วิธีการนับจำนวน Token ของ LLM Model
- การใช้งาน Multi-turn RAG
- การใช้งาน Guardrails
- การสร้าง Text-to-SQL Application

## สิ่งที่คาดหวัง
- สามารถสร้าง RAG ที่สามารถกรองประเภทของคำถามได้
- สามารถสร้าง Chatbot ที่เข้าถึง Text-to-SQL และ Calculator ได้

# ภาคเช้า

## การใช้งาน OpenAI Client

In [ ]:
!pip install rich

In [ ]:
from rich import print

In [ ]:
from openai import OpenAI # เรียกใช้งาน OpenAI Library

สำหรับการใช้งาน OpenAI Model โดยตรง แก้ไขเฉพาะ API Key เท่านั้น แต่ถ้าต้องการใช้งาน OpenAI API Compatibility จำเป็นต้องแก้ไข base_url ด้วย

In [ ]:
# @title ตัวอย่างการใช้งาน OpenAI
# API_KEY="REDACTED_OPENAI_API_KEY" # ใช้งาน GPT-5 หรือ GPT ตัวอื่น ๆ
# client = OpenAI(
#     api_key = API_KEY,
# )
# base_model = "gpt-5.2"

In [ ]:
# @title การใช้งาน Open source model
API_KEY="float16-r-CT1EIdtNcJDOw015AAHj5XSlYKyn" # สำหรับใช้งานผ่าน Float16, Open router, Ollama หรือ Self-host ตัวอื่น ๆ
client = OpenAI(
    api_key = API_KEY,
    base_url="https://proxy-instance.float16.cloud/7188aae3-1e71-4d7c-a6ae-f50ce9cf1983/3900/v1"
)
base_model = "/model/Qwen/Qwen3-VL-30B-A3B-Instruct-FP8"

`chat.completion.create` คือการเรียกใช้งาน model โดยมี arguments สำคัญได้แก่
1. `model` ใช้สำหรับระบุว่าต้องการใช้งาน model ตัวไหน
2. `messages` ใช้สำหรับระบุ prompt และการสนทนาแบบ multi-turn
3. `max_tokens` ใช้สำหรับระบุความยาวสูงสุดสำหรับการตอบกลับ
4. `stream` ใช้ระบุว่า Response ควรตอบกลับมาเป็น Streaming หรือ Non-Streaming

In [ ]:
# @title ทดลองใช้งาน hello world
res = client.chat.completions.create(
    model=base_model,
    messages=[
        {
            "role": "user",
            "content": "สวัสดี คุณทำอะไรได้บ้าง"
        },
    ],
    stream=False
)
print(res)

ChatCompletion(
    id='chatcmpl-D7erOQuFhJfi89lT0hd1NtGWeeuPJ',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='ผมช่วยได้หลายอย่าง เช่น\n\n- ตอบคำถาม/อธิบายความรู้: วิทย์ คณิต ประวัติศาสตร์ กฎหมายทั่วไป ฯลฯ  \n-
แปลภาษาและช่วยเขียน: ไทย–อังกฤษและอื่น ๆ ปรับสำนวน ตรวจไวยากรณ์ สรุป/ขยายความ  \n- ช่วยทำงานเอกสาร: สรุปบทความ ทำโน้ต
ทำโครงร่าง รายงาน อีเมล เรซูเม่  \n- ไอเดียและวางแผน: ตั้งเป้าหมาย แผนอ่านหนังสือ ทริปท่องเที่ยว คอนเทนต์  \n- ช่วยเขียนโค้ด/ดีบัก:
Python, JavaScript, SQL ฯลฯ อธิบายโค้ดได้  \n- วิเคราะห์ข้อมูลแบบทั่วไป: ช่วยคิดสูตร ตาราง วิธีจัดระเบียบข้อมูล  \n- ช่วยจากรูปภาพ:
อธิบายสิ่งที่เห็น อ่านข้อความในภาพ (OCR) ช่วยแก้โจทย์จากภาพ\n\nอยากให้ช่วยเรื่องไหนเป็นพิเศษครับ? ถ้าบอกเป้าหมายและบริบทนิดหน่อย
ผมจะช่วยได้ตรงจุดมากขึ้น',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1770717734,
    model='gpt-5.2-2025-12-11',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=264,
        prompt_tokens=16,
        total_tokens=280,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

## Token

นับจำนวน Token

In [ ]:
response = client.chat.completions.create(
    model=base_model,
    messages=[
        {
            "role": "user",
            "content": "สวัสดี"
        },
    ],
    stream=False
)
print(response)

ChatCompletion(
    id='chatcmpl-D7erVQLnYPDHhcQDU2yabsyfgizjd',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='สวัสดีครับ/ค่ะ 😊 ต้องการให้ช่วยเรื่องอะไรดีครับ/คะ?',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1770717741,
    model='gpt-5.2-2025-12-11',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=23,
        prompt_tokens=10,
        total_tokens=33,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

## Multi-turn RAG

In [ ]:
# @title def store_content_with_vector
from typing import List, Dict, Any, Tuple, Optional
import math

collection = {}

def store_content_with_vector(content_and_vector, collection_key):
    """
    Store content and vector data under a specific key in the collection.

    Args:
        content_and_vector: The data to store. Expected structure:
            - Can be any object (dict, list, tuple, string, etc.)
            - Commonly used as a dict with keys like:
                - "content": (str, list, dict) - the actual content
                - "vector": (list, tuple, np.ndarray) - numerical vector representation
                - "metadata": (dict, optional) - additional info (e.g., source, timestamp)
            Example:
                {
                    "content": "This is a document",
                    "vector": [0.1, 0.2, 0.3]
                }
            - Can also be a simple string, list, or any serializable object.

        collection_key (str): The key under which to store the data. Must be a string.

    Returns:
        dict: Result containing success status and message
    """
    try:
        if collection_key in collection:
            # Append content_and_vector to existing key
            collection[collection_key].append(content_and_vector)
            return {
                "success": True,
                "message": f"Content and vector successfully appended to existing key '{collection_key}'"
            }
        else:
            # Create new key with content_and_vector as a list
            collection[collection_key] = [content_and_vector]
            return {
                "success": True,
                "message": f"New key '{collection_key}' created and content/vector stored"
            }
    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error occurred during storage: {str(e)}"
        }

def clear_collection():
  global collection
  collection = {}

In [ ]:
# @title def get_content
from typing import List, Dict, Any, Tuple, Optional
import math

def get_content(
    query_vector: List[float],
    top_k: int = 5,
    threshold: float = 0.0,
    collection_key: str = None,
    return_vector = False
) -> List[Dict[str, Any]]:
    global collection
    """
    Find the most similar content entries to a query vector using cosine similarity.

    Args:
        query_vector (List[float]): The query vector to search for (must be non-empty)
        top_k (int): Maximum number of results to return (default: 5)
        threshold (float): Minimum similarity score threshold (0.0 to 1.0, default: 0.0)
        collection_key (str, optional): Specific key in collection to search. If None, searches all keys.

    Returns:
        List[Dict[str, Any]]: List of matching results with:
            - "content": The stored content
            - "vector": The stored vector
            - "similarity": Cosine similarity score (0.0 to 1.0)
            - "metadata": Any additional metadata (if present)
            - "collection_key": The key where this entry was found

    Raises:
        ValueError: If query_vector is empty or invalid
        TypeError: If parameters have wrong types
        RuntimeError: If no data is available for searching
    """
    try:
        # Input validation
        if not isinstance(query_vector, (list, tuple)):
            return {
                "success": False,
                "error": f"query_vector must be a list or tuple, got {type(query_vector).__name__}"
            }

        if not query_vector:
            return {
                "success": False,
                "error": "query_vector cannot be empty"
            }

        # Check if collection exists and has data
        if not collection:
            return {
                "success": False,
                "error": "Collection is empty. No data to search."
            }

        # Determine which keys to search
        keys_to_search = [collection_key] if collection_key else collection.keys()

        # Validate that the specified key exists if provided
        if collection_key and collection_key not in collection:
            return {
                "success": False,
                "error": f"collection_key '{collection_key}' does not exist in the collection"
            }

        # List to store results
        results = []
        # Search through all relevant keys
        for key in keys_to_search:
            if not isinstance(collection[key], list):
                continue  # Skip non-list entries

            for item in collection[key]:
                # Extract vector from item
                if isinstance(item, dict):
                    vector_data = item.get("vector")
                    content = item.get("content", item)  # Use content field or the item itself
                    metadata = item.get("metadata", {})
                elif isinstance(item, (list, tuple)):
                    vector_data = item
                    content = item
                    metadata = {}
                else:
                    # Handle other types (string, number, etc.)
                    vector_data = None
                    content = item
                    metadata = {}

                # Skip if no vector data available
                if vector_data is None or not isinstance(vector_data, (list, tuple)):
                    continue

                # Ensure vector dimensions match
                if len(query_vector) != len(vector_data):
                    continue

                # Calculate cosine similarity
                try:
                    # Calculate dot product
                    dot_product = sum(a * b for a, b in zip(query_vector, vector_data))

                    # Calculate magnitudes
                    query_magnitude = math.sqrt(sum(a * a for a in query_vector))
                    vector_magnitude = math.sqrt(sum(b * b for b in vector_data))

                    # Avoid division by zero
                    if query_magnitude == 0 or vector_magnitude == 0:
                        similarity = 0.0
                    else:
                        similarity = dot_product / (query_magnitude * vector_magnitude)

                    # Apply threshold
                    if similarity < threshold:
                        continue


                    if return_vector :
                      # Add to results
                      results.append({
                          "content": content,
                          "vector": vector_data,
                          "similarity": round(similarity, 6),
                          "metadata": metadata,
                          "collection_key": key
                      })
                    else :
                      results.append({
                          "content": content,
                          "similarity": round(similarity, 6),
                          "metadata": metadata,
                          "collection_key": key
                      })

                except (TypeError, ValueError, OverflowError) as e:
                    # Skip items with calculation errors
                    continue

        # Sort by similarity (descending) and take top_k
        results.sort(key=lambda x: x["similarity"], reverse=True)
        top_results = results[:top_k]

        # Return results
        return {
            "success": True,
            "results": top_results,
            "total_found": len(results),
            "top_k": top_k,
            "threshold": threshold
        }

    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error occurred during search: {str(e)}"
        }

In [ ]:
! pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.2 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.3.5-py3-none-any.whl size=233746 sha256=61e2f16ce85ae7b4b1b5f616606290a025766cb7d15400bb3756509bf1b9f877
  Stored in directory: /root/.cache/pip/wheels/b2/1f/f6/78f862bb80cb959cc9960b7c4e2d1f702b1bc0e79d19b5f124
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=6317b5432cfc1923924f

In [ ]:
! pip install transformers==4.57.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 34.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.3.7
    Uninstalling huggingface_hub-1.3.7:
      Successfully uninstalled huggingface_hub-1.3.7
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


In [ ]:
from FlagEmbedding import BGEM3FlagModel
import numpy as np

In [ ]:
# @title Init RAG
## Insert

documents = [
    "ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม",
    "กะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา น้ำตาล และพริกขี้หนู",
    "ก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส",
    "ผัดไท - ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล"
]
def do_vector_by_yourself(documents):

  model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
  for doc in documents :
      embedded_result = model.encode(doc,
                                  batch_size=1,
                                  max_length=8192,
                                  )['dense_vecs']
      store_content_with_vector({
          "content" : doc,
          "vector" : list(embedded_result)
      },'my_collection')

do_vector_by_yourself(documents)

## Search

user_prompt = "แนะนำเมนูที่ทำมาจากเส้นหน่อย"

def do_embedding_by_yourself(user_prompt):
  user_query = None
  model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
  user_query = model.encode(user_prompt,
                              batch_size=1,
                              max_length=8192,
                              )['dense_vecs']
  return user_query

query_vector = do_embedding_by_yourself(user_prompt)

print(query_vector, type(query_vector))
search_result = get_content(query_vector = list(query_vector), collection_key = "my_collection")
print(search_result)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

...

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 2065.14it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 60.02it/s]


[-0.03102   0.03128  -0.04706  ... -0.00384   0.00364  -0.000893] <class 'numpy.ndarray'>

/tmp/ipython-input-1470327191.py:129: RuntimeWarning: overflow encountered in cast
  "similarity": round(similarity, 6),
/tmp/ipython-input-1470327191.py:129: RuntimeWarning: invalid value encountered in divide
  "similarity": round(similarity, 6),


{'success': True,
 'results': [{'content': 'ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม',
   'similarity': np.float16(nan),
   'metadata': {},
   'collection_key': 'my_collection'},
  {'content': 'กะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา น้ำตาล และพริกขี้หนู',
   'similarity': np.float16(nan),
   'metadata': {},
   'collection_key': 'my_collection'},
  {'content': 'ก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส',
   'similarity': np.float16(nan),
   'metadata': {},
   'collection_key': 'my_collection'},
  {'content': 'ผัดไท - ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล',
   'similarity': np.float16(nan),
   'metadata': {},
   'collection_key': 'my_collection'}],
 'total_found': 4,
 'top_k': 5,
 'threshold': 0.0}

In [ ]:
retrieved_content = ""
for item in search_result["results"] :
    content = item['content']
    retrieved_content += content + "\n"

retrieved_content

'ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม\nกะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา น้ำตาล และพริกขี้หนู\nก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส\nผัดไท - ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล\n'

In [ ]:
user_prompt = "แนะนำเมนูที่ทำมาจากเส้นหน่อย"
messages = [
        {
            "role": "user",
            "content": f"""
            ===========================CONTEXT===========================
            Context : {retrieved_content}
            ===========================CONTEXT===========================

            User question : {user_prompt}
            """
        }
]
messages

[{'role': 'user',
  'content': '\n            ===========================CONTEXT===========================\n            Context : ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม\nกะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา น้ำตาล และพริกขี้หนู\nก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส\nผัดไท - ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล\n\n            ===========================CONTEXT===========================\n\n            User question : แนะนำเมนูที่ทำมาจากเส้นหน่อย\n            '}]

In [ ]:
response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)

In [ ]:
assistant_response = response.choices[0].message.content
print(assistant_response)

เมนูที่ทำมาจาก “เส้น” จากข้อมูลที่ให้มา มี 2 อย่างนี้:

- **ก๋วยเตี๋ยว** — ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุป ใส่เนื้อสัตว์ ผัก และเครื่องปรุง
- **ผัดไท** — ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และปรุงด้วยน้ำปลา น้ำมะนาว น้ำตาล

In [ ]:
messages.append({
    "role" : "assistant",
    "content" : assistant_response
})

In [ ]:
messages.append({
    "role" : "user",
    "content" : "แล้วที่เหลือแหละ"
})

In [ ]:
print(messages)

[
    {
        'role': 'user',
        'content': '\n            ===========================CONTEXT===========================\n            
Context : ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม\nกะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา
น้ำตาล และพริกขี้หนู\nก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส\nผัดไท -
ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล\n\n            
===========================CONTEXT===========================\n\n            User question : 
แนะนำเมนูที่ทำมาจากเส้นหน่อย\n            '
    },
    {
        'role': 'assistant',
        'content': 'เมนูที่ทำมาจาก “เส้น” จากข้อมูลที่ให้มา มี 2 อย่างนี้:\n\n- **ก๋วยเตี๋ยว** — ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม
ผัดหรือต้มในน้ำซุป ใส่เนื้อสัตว์ ผัก และเครื่องปรุง\n- **ผัดไท** — ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และปรุงด้วยน้ำปลา น้ำมะนาว น้ำตาล'
    },
    {'role': 'user', 'content': 'แล้วที่เหลือแหละ'}
]

In [ ]:
response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)
assistant_response = response.choices[0].message.content
print(assistant_response)

ที่เหลือ (ที่ไม่ใช่เมนูเส้น) คือ:

- **ข้าวมันไก่** — ทำจากข้าวสวยต้มในน้ำซุปไก่ เสิร์ฟกับไก่ต้ม น้ำจิ้ม  
- **กะเพรา** — ข้าวสวยราดด้วยผัดกะเพรา (ไก่หรือหมู) ปรุงด้วยน้ำปลา น้ำตาล และพริกขี้หนู

In [ ]:
print(response)

ChatCompletion(
    id='chatcmpl-D7evFpoaOXre8u1SX9OeWY5nXiKjp',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='ที่เหลือ (ที่ไม่ใช่เมนูเส้น) คือ:\n\n- **ข้าวมันไก่** — ทำจากข้าวสวยต้มในน้ำซุปไก่ เสิร์ฟกับไก่ต้ม น้ำจิ้ม  \n-
**กะเพรา** — ข้าวสวยราดด้วยผัดกะเพรา (ไก่หรือหมู) ปรุงด้วยน้ำปลา น้ำตาล และพริกขี้หนู',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1770717973,
    model='gpt-5.2-2025-12-11',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=104,
        prompt_tokens=358,
        total_tokens=462,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

## Guardrails แบบ LLM Model

ทดลองใช้ Model Guardrails สำหรับการป้องกัน prompt ในหัวข้อที่ไม่พึงประสงค์และอันตราย

โดยทดลองใช้ผ่าน Model Qwen3Guard ซึ่งรองรับการใช้งานภาษาไทยแบบ 100%

In [ ]:
# @title Def function สำหรับ Guardrails

from transformers import AutoModelForCausalLM, AutoTokenizer
import re
model_name = "Qwen/Qwen3Guard-Gen-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)


def extract_label_and_categories(content):
    safe_pattern = r"Safety: (Safe|Unsafe|Controversial)"
    category_pattern = r"(Violent|Non-violent Illegal Acts|Sexual Content or Sexual Acts|PII|Suicide & Self-Harm|Unethical Acts|Politically Sensitive Topics|Copyright Violation|Jailbreak|None)"
    safe_label_match = re.search(safe_pattern, content)
    label = safe_label_match.group(1) if safe_label_match else None
    categories = re.findall(category_pattern, content)
    return label, categories

def guardrails(prompt):
  # prepare the model input
  # for prompt moderation
  messages = [
      {"role": "user", "content": prompt}
  ]
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  # conduct text completion
  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=128
  )
  output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

  content = tokenizer.decode(output_ids, skip_special_tokens=True)
  safe_label, categories = extract_label_and_categories(content)
  return content

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

In [ ]:
prompt = "ทำยานอนหลับ สำหรับเครื่องดื่มแอลกอฮอล์ทำยังไง"
print(guardrails(prompt))

Safety: Unsafe
Categories: Non-violent Illegal Acts

## Guardrails แบบ Vector


In [ ]:
bge_m3_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
prompt = "คุณคือผู้ช่วยด้านประกันภัยที่ช่วยเขียนโค้ดได้, ช่วยเขียน python hello world ให้หน่อย"
allow_topic = ["ประกันภัย", "แผนประกัน", "เคลมประกัน"]

prompt_vector = bge_m3_model.encode(prompt,
                            batch_size=1,
                            max_length=8192,
                            )['dense_vecs']
allow_topic_vector = bge_m3_model.encode(allow_topic,
                            batch_size=4,
                            max_length=8192,
                            )['dense_vecs']

similarity = prompt_vector @ allow_topic_vector.T



similarity

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 1544.29it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 62.64it/s]


array([0.5273, 0.4983, 0.528 ], dtype=float16)

## ภาคบ่าย

## สร้าง SQL Databas ด้วย SQLite

In [ ]:
%load_ext sql

In [ ]:
%%sql
sqlite:///new_records.db

### สร้าง table ชื่อ symptoms

โดยประกอบด้วย 3 fields
1. date
2. symptom
3. name

In [ ]:
%%sql
CREATE TABLE symptoms (
    date DATE NOT NULL,
    symptom VARCHAR(100) NOT NULL,
    name VARCHAR(100) NOT NULL
);

 * sqlite:///new_records.db
Done.


[]

## Insert data ตัวอย่างเข้าไป

In [ ]:
%%sql
INSERT INTO symptoms (date, symptom, name) VALUES
('2025-08-10', 'กล้ามเนื้ออักเสบ', 'มติชน มณีกาศ'),
('2025-08-10', 'กระดูกหัก', 'มติชน มณีกาศ'),
('2025-09-10', 'กล้ามเนื้ออักเสบ', 'สมชาย');

 * sqlite:///new_records.db
3 rows affected.


[]

## ลอง Connect Database ด้วย python

In [ ]:
import sqlite3
import json
# เชื่อมต่อกับฐานข้อมูล SQLite เดิม (ไฟล์ชื่อ new_records.db)
conn = sqlite3.connect('new_records.db')
cursor = conn.cursor()

# รันคำสั่ง SELECT เพื่อดึงข้อมูลทั้งหมดจากตาราง symptoms
print("กำลังดึงข้อมูลจากตาราง symptoms...")
cursor.execute('SELECT * FROM symptoms')

# ดึงผลลัพธ์ทั้งหมด
rows = cursor.fetchall()

# แสดงผล
if rows:
    print("\nข้อมูลในตาราง symptoms:")
    for row in rows:
        print(f"วันที่: {row[0]}, อาการ: {row[1]}, ชื่อ: {row[2]}")
else:
    print("ไม่มีข้อมูลในตาราง symptoms")

# ปิดการเชื่อมต่อ
conn.close()
print("\n✅ การเชื่อมต่อปิดเรียบร้อย")

กำลังดึงข้อมูลจากตาราง symptoms...

ข้อมูลในตาราง symptoms:

วันที่: 2025-08-10, อาการ: กล้ามเนื้ออักเสบ, ชื่อ: มติชน มณีกาศ

วันที่: 2025-08-10, อาการ: กระดูกหัก, ชื่อ: มติชน มณีกาศ

วันที่: 2025-09-10, อาการ: กล้ามเนื้ออักเสบ, ชื่อ: สมชาย

✅ การเชื่อมต่อปิดเรียบร้อย

## ดึงค่า Table schema เพื่อเป็น Context ให้กับ LLM

In [ ]:
conn = sqlite3.connect('new_records.db') #1
cursor = conn.cursor() #2

cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='symptoms'")
result = cursor.fetchone()
if result:
    ddl = result[0]
    print(ddl)

CREATE TABLE symptoms (
    date DATE NOT NULL,
    symptom VARCHAR(100) NOT NULL,
    name VARCHAR(100) NOT NULL
)

## สร้าง prompt ให้กับ LLM ให้รองรับการ Generate SQL

In [ ]:
# @title system prompt แบบทั่วไป
system_prompt = f"""You are a specialized AI assistant expert in database systems, designed to help users generate correct, secure, and efficient SQL statements.

=================
General knowledge
Today is 6/Feb/2026
Location is Bangkok, Thailand
=================

=================
Table schema
CREATE TABLE symptoms (
    date DATE NOT NULL,
    symptom VARCHAR(100) NOT NULL,
    name VARCHAR(100) NOT NULL
)
=================
You must follow these guidelines strictly:

Understand the Request Clearly

Carefully analyze the user’s request to determine the exact data needed, filtering conditions, grouping logic, or database operations (e.g., SELECT, INSERT, UPDATE, DELETE).
If any details are missing (e.g., table names, column names, date ranges, relationships), ask clarifying questions before generating SQL.
Generate Valid and Standard SQL

Write syntactically correct SQL that adheres to SQL-92 standards or the specific database system (e.g., MySQL, PostgreSQL, SQL Server, SQLite) if known.
Use appropriate SQL clauses: SELECT, FROM, JOIN, WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, INSERT INTO, UPDATE, DELETE, CREATE TABLE, etc.
Prioritize Security

Never generate SQL that is vulnerable to SQL injection.
Do not concatenate user input directly into SQL queries.
Avoid unsafe functions like EXEC, sp_executesql, or dynamic SQL unless explicitly requested and properly parameterized.
Ensure Readability and Best Practices

Format SQL with proper indentation, line breaks, and capitalization (e.g., SELECT, FROM, WHERE in uppercase).
Use meaningful aliases for tables (e.g., c for customers, o for orders).
Organize the query logically: SELECT → FROM → JOIN → WHERE → GROUP BY → HAVING → ORDER BY → LIMIT.
Provide Clear Explanations

Include a brief explanation of what the SQL query does.
Highlight key conditions, aggregations, or join logic.
Mention any assumptions made (e.g., “Assuming orders table has order_date and total_amount columns”).
Ask for Clarification When Needed

If the schema (tables, columns, relationships) is unknown, request:
Table names
Column names and data types
Foreign key relationships
Sample data or example use case
"""

In [ ]:
# @title
messages = [{
  "role" : "system",
  "content" : system_prompt
},{
  "role" : "user",
  "content" : "อาการไหนยอดฮิตบ้าง"
}]

response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)

print(response.choices[0].message.content)

```sql
SELECT
    s.symptom,
    COUNT(*) AS report_count
FROM symptoms AS s
GROUP BY
    s.symptom
ORDER BY
    report_count DESC,
    s.symptom ASC;
```

คำอธิบาย: คิวรีนี้นับจำนวนครั้งที่มีการบันทึกแต่ละ `symptom` แล้วเรียงจาก “ยอดฮิต” (จำนวนมากสุด) ลงมาน้อยสุด

ต้องการ “ยอดฮิตช่วงไหน” ไหมครับ (เช่น 7 วันล่าสุด/เดือนนี้/ปีนี้) และอยากแยกตาม `name` หรือรวมทุกคน?

In [ ]:
# @title system prompt แบบ Agentic
system_prompt = f"""You are a specialized AI assistant expert in database systems, designed to help users generate correct, secure, and efficient SQL statements.

=================
General knowledge
Today is 6/Feb/2025
Location is Bangkok, Thailand
=================

=================
Table schema
{ddl}
=================
You must follow these guidelines strictly:

You MUST reponse start with ```sql and DON'T include any comment or explain anythings.
Only contain SQL statement.
If you cant generate the SQL statement please return <INSUFFICIENT_DATA>

Understand the Request Clearly

Carefully analyze the user’s request to determine the exact data needed, filtering conditions, grouping logic, or database operations (e.g., SELECT, INSERT, UPDATE, DELETE).
If any details are missing (e.g., table names, column names, date ranges, relationships), ask clarifying questions before generating SQL.
Generate Valid and Standard SQL

Write syntactically correct SQL that adheres to SQL-92 standards or the specific database system (e.g., MySQL, PostgreSQL, SQL Server, SQLite) if known.
Use appropriate SQL clauses: SELECT, FROM, JOIN, WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, INSERT INTO, UPDATE, DELETE, CREATE TABLE, etc.
Prioritize Security

Never generate SQL that is vulnerable to SQL injection.
Do not concatenate user input directly into SQL queries.
Avoid unsafe functions like EXEC, sp_executesql, or dynamic SQL unless explicitly requested and properly parameterized.
Ensure Readability and Best Practices

Format SQL with proper indentation, line breaks, and capitalization (e.g., SELECT, FROM, WHERE in uppercase).
Use meaningful aliases for tables (e.g., c for customers, o for orders).
Organize the query logically: SELECT → FROM → JOIN → WHERE → GROUP BY → HAVING → ORDER BY → LIMIT.
Provide Clear Explanations

Include a brief explanation of what the SQL query does.
Highlight key conditions, aggregations, or join logic.
Mention any assumptions made (e.g., “Assuming orders table has order_date and total_amount columns”).
Ask for Clarification When Needed

If the schema (tables, columns, relationships) is unknown, request:
Table names
Column names and data types
Foreign key relationships
Sample data or example use case
"""

In [ ]:
messages = [{
  "role" : "system",
  "content" : system_prompt
},{
  "role" : "user",
  "content" : "อาการไหนยอดฮิตบ้าง"
}]

response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)

print(response.choices[0].message.content)

```sql
SELECT
  s.symptom,
  COUNT(*) AS occurrence_count
FROM symptoms AS s
GROUP BY
  s.symptom
ORDER BY
  occurrence_count DESC,
  s.symptom ASC;```

In [ ]:
messages = [{
  "role" : "system",
  "content" : system_prompt
},{
  "role" : "user",
  "content" : "ทำไมคนเคลมเยอะจัง"
}]

response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)

print(response.choices[0].message.content)

<INSUFFICIENT_DATA>

## รวม LLM Chatbot เข้ากับ Text-to-SQL และ Calculator

In [ ]:
def calculator(num1, operator, num2):
    if operator == '+':
        return num1 + num2
    elif operator == '-':
        return num1 - num2
    elif operator == '*':
        return num1 * num2
    elif operator == '/':
        if num2 == 0:
            return "Error: Divided by Zero!"
        return num1 / num2
    elif operator == '**' or operator == '^':
        return num1 ** num2
    else:
        return f"Error: Not support '{operator}'"

In [ ]:
system_prompt = """You are a helpful AI assistant capable of interacting with two specialized tools to assist users with calculations and database queries. Always use the appropriate tool when needed, and only when the task requires it.

================
Database schema

CREATE TABLE symptoms (
    date DATE NOT NULL,
    symptom VARCHAR(100) NOT NULL,
    name VARCHAR(100) NOT NULL
)

================

Available tools:
1. Calculator: Perform arithmetic operations.
   - Arguments:
     - num_1: First number (float or integer)
     - operator: Arithmetic operator (+, -, *, /)
     - num_2: Second number (float or integer)
   - Use this tool when the user asks for a mathematical calculation.

2. Text-to-SQL: Convert natural language questions into SQL queries.
   - Argument:
     - user_question: The user's question in natural language (e.g., "What is the average salary of employees in the engineering department?")
   - Use this tool when the user asks a question that requires querying a database.

Tool invocation format:
Always use the following JSON format to invoke a tool:
{
  "name": "tool_name",
  "arguments": { "key": "value" }
}

Examples:
- For calculation: {"name": "Calculator", "arguments": {"num_1": 10, "operator": "+", "num_2": 5}}
- For Text-to-SQL: {"name": "Text-to-SQL", "arguments": {"user_question": "Find all customers from New York"}}

When responding:
- ALWAYS use calculation when user ask about math, use the Calculator tool with the correct arguments.
- If the task requires querying a database using a natural language question, use the Text-to-SQL tool.


Do not make up answers. Only use tools when necessary. Be precise and concise."""

In [ ]:
def sql_generate(prompt):
  system_prompt = f"""You are a specialized AI assistant expert in database systems, designed to help users generate correct, secure, and efficient SQL statements.

  =================
  General knowledge
  Today is 6/Feb/2025
  Location is Bangkok, Thailand
  =================

  =================
  Table schema
  {ddl}
  =================
  You must follow these guidelines strictly:

  You MUST reponse start with ```sql and DON'T include any comment or explain anythings. Only contain SQL statement.
  If you cant generate the SQL statement please return <INSUFFICIENT_DATA>

  Understand the Request Clearly

  Carefully analyze the user’s request to determine the exact data needed, filtering conditions, grouping logic, or database operations (e.g., SELECT, INSERT, UPDATE, DELETE).
  If any details are missing (e.g., table names, column names, date ranges, relationships), ask clarifying questions before generating SQL.
  Generate Valid and Standard SQL

  Write syntactically correct SQL that adheres to SQL-92 standards or the specific database system (e.g., MySQL, PostgreSQL, SQL Server, SQLite) if known.
  Use appropriate SQL clauses: SELECT, FROM, JOIN, WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, INSERT INTO, UPDATE, DELETE, CREATE TABLE, etc.
  Prioritize Security

  Never generate SQL that is vulnerable to SQL injection.
  Do not concatenate user input directly into SQL queries.
  Avoid unsafe functions like EXEC, sp_executesql, or dynamic SQL unless explicitly requested and properly parameterized.
  Ensure Readability and Best Practices

  Format SQL with proper indentation, line breaks, and capitalization (e.g., SELECT, FROM, WHERE in uppercase).
  Use meaningful aliases for tables (e.g., c for customers, o for orders).
  Organize the query logically: SELECT → FROM → JOIN → WHERE → GROUP BY → HAVING → ORDER BY → LIMIT.
  Provide Clear Explanations

  Include a brief explanation of what the SQL query does.
  Highlight key conditions, aggregations, or join logic.
  Mention any assumptions made (e.g., “Assuming orders table has order_date and total_amount columns”).
  Ask for Clarification When Needed

  If the schema (tables, columns, relationships) is unknown, request:
  Table names
  Column names and data types
  Foreign key relationships
  Sample data or example use case
  """
  messages = [{
  "role" : "system",
  "content" : system_prompt
  },{
    "role" : "user",
    "content" : prompt
  }]

  response = client.chat.completions.create(
      model=base_model,
      messages=messages,
      stream=False
  )

  sql_raw = response.choices[0].message.content

  if sql_raw.startswith("```sql"):
    conn = sqlite3.connect('new_records.db')
    cursor = conn.cursor()
    sql_statement = sql_raw.replace('```sql',"").replace("```","")
    print(sql_statement)
    cursor.execute(sql_statement)
    result = cursor.fetchall()
    return result
  return None

def call_function(function_name, arguments):
  if function_name == "calculator":
    return calculator(arguments["num_1"],arguments["operator"],arguments["num_2"])
  elif function_name == "text-to-sql":
    return sql_generate(arguments["user_question"])

def parse_tools(response):
  try :
    text = response.choices[0].message.content
    text = text.split("\n")[0]
  except :
    print(f"Failed to split newline : raw text is {text}")
    return json.loads(text)

  return json.loads(text)


In [ ]:
messages = [{
  "role" : "system",
  "content" : system_prompt
},{
  "role" : "user",
  "content" : "11.241 * 1472.2832 เท่ากับเท่าไหร่"
}]

response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)

print(response.choices[0].message.content)
# print(response)
tools_call = parse_tools(response)
print(tools_call)
print(call_function(tools_call['name'].lower(), tools_call['arguments']))

{"name": "Calculator", "arguments": {"num_1": 11.241, "operator": "*", "num_2": 1472.2832}}

{'name': 'Calculator', 'arguments': {'num_1': 11.241, 'operator': '*', 'num_2': 1472.2832}}

16549.9354512

In [ ]:
messages = [{
  "role" : "system",
  "content" : system_prompt
},{
  "role" : "user",
  "content" : "อาการไหนยอดฮิตบ้าง"
}]

response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)

print(response.choices[0].message.content)
tools_call = parse_tools(response)
print(tools_call)
print(call_function(tools_call['name'].lower(), tools_call['arguments']))

{"name": "Text-to-SQL", "arguments": {"user_question": "Which symptom is the most popular?"}}

{'name': 'Text-to-SQL', 'arguments': {'user_question': 'Which symptom is the most popular?'}}

SELECT symptom, COUNT(*) as frequency
FROM symptoms
GROUP BY symptom
ORDER BY frequency DESC
LIMIT 1;

[('กล้ามเนื้ออักเสบ', 2)]